# Prepare CNN Data

## Setup

In [1]:
# USE ACCOUNT: robmarty3@gmail.com
import ee
ee.Authenticate()
ee.Initialize()

Enter verification code:  4/1AX4XfWjvzeEa_F_HPcf3O1ySQXIx0mZGcNzjQdU44WmXjP8hgeoag5WXPD4



Successfully saved authorization token.


In [2]:
import numpy as np
import os, datetime
import pandas as pd
import itertools
import tensorflow as tf
import config as cf
import ee_utils as utils
import eeconvert
import time

#import geetools
#from geetools import ui, cloud_mask
#cloud_mask_landsatSR = cloud_mask.landsatSR()
#cloud_mask_sentinel2 = cloud_mask.sentinel2()

/Users/robmarty/anaconda3/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}
/Users/robmarty/anaconda3/lib/python3.7/site-packages/xarray/core/merge.py:17: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  PANDAS_TYPES = (pd.Series, pd.DataFrame, pd.Panel)
/Users/robmarty/anaconda3/lib/python3.7/site-packages/distributed/config.py:20: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  defaults = yaml.load(f)


## Parameters

In [3]:
#224
SURVEY_NAME = 'DHS'
SATELLITE = 'l8'
SKIP_IF_SCRAPED = True
CHUNK_SIZE = 10 # Number of observtaions to scrape in GEE at any given time
DROPBOX_DIR = cf.DROPBOX_DIRECTORY
GOOGLEDRIVE_DIR = cf.GOOGLEDRIVE_DIRECTORY

if SATELLITE == 's2':
    KERNEL_SIZE = 500
elif SATELLITE == 'l8':
    KERNEL_SIZE = 167

print(KERNEL_SIZE)

167


In [4]:
# Directory to store tfrecords
out_path = os.path.join(GOOGLEDRIVE_DIR, 
            'Data', 
            SURVEY_NAME, 
            'FinalData',
            'Individual Datasets',
            'cnn_' + SATELLITE,
            'tfrecords')

## Load Data

In [ ]:
# Relies on VIIRs Values
survey_df = pd.read_csv(os.path.join(DROPBOX_DIR, 'Data', SURVEY_NAME, 'FinalData', 'Individual Datasets', 'data_for_cnn.csv'))

In [ ]:
# Remove observations that had issues
#survey_df = survey_df[survey_df['GID_2'] != "IND.18.1_1"]
survey_df = survey_df[survey_df['uid'] != 'CO201000003682']
survey_df = survey_df[survey_df['uid'] != 'CO201000003683']
survey_df = survey_df[survey_df['uid'] != 'CO201000004885']
survey_df = survey_df[survey_df['uid'] != 'IA201400180058']
survey_df = survey_df[survey_df['uid'] != 'IA201400180040']
survey_df = survey_df[survey_df['uid'] != 'IA201400300032']
survey_df = survey_df[survey_df['uid'] != 'IA201400010147']
survey_df = survey_df[survey_df['uid'] != 'NM201300000001']
survey_df = survey_df[survey_df['uid'] != 'CO201000002924']
survey_df = survey_df[survey_df['uid'] != 'CO201000002524']
survey_df = survey_df[survey_df['uid'] != 'CO201000001170']
survey_df = survey_df[survey_df['uid'] != 'CO201000003554']
survey_df = survey_df[survey_df['uid'] != 'GA201200000310']
survey_df = survey_df[survey_df['uid'] != 'IA201400180104'] # maybe OK?
survey_df = survey_df[survey_df['uid'] != 'IA201400180012']
survey_df = survey_df[survey_df['uid'] != 'IA201400180011']
survey_df = survey_df[survey_df['uid'] != 'IA201400180048']
survey_df = survey_df[survey_df['uid'] != 'IA201400300002']
survey_df = survey_df[survey_df['uid'] != 'IA201400130192']
survey_df = survey_df[survey_df['uid'] != 'IA201400140845']
survey_df = survey_df[survey_df['uid'] != 'IA201400010076']

# List of TF Records
tf_record_list = list(np.unique(survey_df.tfrecord_name))

In [ ]:
# If skip already scraped, remove existing tfrecords from tf_record_list
if SKIP_IF_SCRAPED:
    tf_records_exist = os.listdir(out_path)
    tf_record_list = [x for x in tf_record_list if x not in tf_records_exist]

In [ ]:
### Loop through all tfrecords
for tfr_i in tf_record_list:
    
    # Sometimes we get computational time out errors. If occurs, just skip and go to next.
    # We can then go back and rescrape missed ones.
    try:
    
        survey_df_yeari = survey_df[survey_df['tfrecord_name'] == tfr_i]
        year_i = survey_df_yeari['year'].iloc[0]

        ### Loop through chunks within tfrecord (can only pull so much data from GEE at a time)
        survey_df_yeari['chunk_id'] = utils.chunk_ids(survey_df_yeari.shape[0], CHUNK_SIZE)

        print("Putting " + str(survey_df_yeari.shape[0]) + " observations into " + tfr_i)

        proto_examples_all = []
        for chunk_i in list(np.unique(survey_df_yeari.chunk_id)):
            time.sleep(6)
            print("Observation: " + str(len(proto_examples_all)) + "/" + str(survey_df_yeari.shape[0]))

            survey_df_yeari_chunki = survey_df_yeari[survey_df_yeari['chunk_id'] == chunk_i]

            proto_examples_i = utils.prep_cnn_np(survey_df_yeari_chunki, SATELLITE, KERNEL_SIZE, year_i)
            proto_examples_all.extend(proto_examples_i)

        ### Save data as tf record
        out_path_i = os.path.join(out_path, tfr_i)
        print(out_path_i)
        with tf.io.TFRecordWriter(out_path_i) as writer:
            for tf_example in proto_examples_all:
                writer.write(tf_example.SerializeToString())

        print("Success \o/")
                
    except:
        pass

/Users/robmarty/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':


Putting 100 observations into forcnn_IA_5_37_all.tfrecord
Observation: 0/100
Observation: 10/100
Observation: 20/100
Observation: 30/100
Observation: 40/100
Observation: 50/100
Observation: 60/100
Observation: 70/100
Observation: 80/100
Observation: 90/100
/Users/robmarty/Google Drive/World Bank/IEs/Pakistan Poverty Estimation/Data/DHS/FinalData/Individual Datasets/cnn_l8/tfrecords/forcnn_IA_5_37_all.tfrecord
Success \o/
Putting 100 observations into forcnn_IA_5_38_all.tfrecord
Observation: 0/100
Observation: 10/100
Observation: 20/100
Observation: 30/100
Observation: 40/100
Observation: 50/100
Observation: 60/100
Observation: 70/100
Observation: 80/100
Observation: 90/100
/Users/robmarty/Google Drive/World Bank/IEs/Pakistan Poverty Estimation/Data/DHS/FinalData/Individual Datasets/cnn_l8/tfrecords/forcnn_IA_5_38_all.tfrecord
Success \o/
Putting 100 observations into forcnn_IA_5_39_all.tfrecord
Observation: 0/100
Observation: 10/100
Observation: 20/100
Observation: 30/100
Observation: 

/Users/robmarty/Documents/Github/Pakistan-Poverty-from-Sky/DataWork/02_get_process_ancillary_data/CNN Features Predict NTL/ee_utils.py:80: RuntimeWarning: invalid value encountered in true_divide
  return (values2 - values1)/(values2 + values1)


Observation: 10/100
Observation: 20/100
Observation: 30/100
Observation: 40/100
Observation: 50/100
Observation: 60/100
Observation: 70/100
Observation: 80/100
Observation: 90/100
/Users/robmarty/Google Drive/World Bank/IEs/Pakistan Poverty Estimation/Data/DHS/FinalData/Individual Datasets/cnn_l8/tfrecords/forcnn_KE_4_1_all.tfrecord
Success \o/
Putting 44 observations into forcnn_KE_4_2_all.tfrecord
Observation: 0/44
Observation: 10/44
Observation: 20/44
Observation: 30/44
Observation: 40/44
/Users/robmarty/Google Drive/World Bank/IEs/Pakistan Poverty Estimation/Data/DHS/FinalData/Individual Datasets/cnn_l8/tfrecords/forcnn_KE_4_2_all.tfrecord
Success \o/
Putting 100 observations into forcnn_KE_5_1_all.tfrecord
Observation: 0/100
Observation: 10/100
Observation: 20/100
Observation: 30/100
Observation: 40/100
Observation: 50/100
Observation: 60/100
Observation: 70/100
Observation: 80/100
Observation: 90/100
/Users/robmarty/Google Drive/World Bank/IEs/Pakistan Poverty Estimation/Data/DHS

W1008 15:31:57.348291 4377071040 http.py:171] Sleeping 0.57 seconds before retry 1 of 5 for request: POST https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/value:compute?prettyPrint=false&alt=json, after 502


Observation: 30/83
Observation: 40/83
Observation: 50/83
Observation: 60/83
Observation: 70/83
Observation: 80/83
/Users/robmarty/Google Drive/World Bank/IEs/Pakistan Poverty Estimation/Data/DHS/FinalData/Individual Datasets/cnn_l8/tfrecords/forcnn_ZM_5_1_all.tfrecord
Success \o/
Putting 38 observations into forcnn_ZW_1_1_all.tfrecord
Observation: 0/38
Observation: 10/38
Observation: 20/38


W1008 15:35:38.584781 4377071040 http.py:171] Sleeping 0.82 seconds before retry 1 of 5 for request: POST https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/value:compute?prettyPrint=false&alt=json, after 502


Observation: 30/38
/Users/robmarty/Google Drive/World Bank/IEs/Pakistan Poverty Estimation/Data/DHS/FinalData/Individual Datasets/cnn_l8/tfrecords/forcnn_ZW_1_1_all.tfrecord
Success \o/
Putting 28 observations into forcnn_ZW_2_1_all.tfrecord
Observation: 0/28
Observation: 10/28
Observation: 20/28
/Users/robmarty/Google Drive/World Bank/IEs/Pakistan Poverty Estimation/Data/DHS/FinalData/Individual Datasets/cnn_l8/tfrecords/forcnn_ZW_2_1_all.tfrecord
Success \o/
Putting 37 observations into forcnn_ZW_3_1_all.tfrecord
Observation: 0/37
Observation: 10/37
Observation: 20/37
Observation: 30/37
/Users/robmarty/Google Drive/World Bank/IEs/Pakistan Poverty Estimation/Data/DHS/FinalData/Individual Datasets/cnn_l8/tfrecords/forcnn_ZW_3_1_all.tfrecord
Success \o/
Putting 99 observations into forcnn_ZW_4_1_all.tfrecord
Observation: 0/99
Observation: 10/99
Observation: 20/99
Observation: 30/99
Observation: 40/99
Observation: 50/99
Observation: 60/99
Observation: 70/99
Observation: 80/99
Observation

/Users/robmarty/Documents/Github/Pakistan-Poverty-from-Sky/DataWork/02_get_process_ancillary_data/CNN Features Predict NTL/ee_utils.py:80: RuntimeWarning: divide by zero encountered in true_divide
  return (values2 - values1)/(values2 + values1)


Observation: 60/63
/Users/robmarty/Google Drive/World Bank/IEs/Pakistan Poverty Estimation/Data/DHS/FinalData/Individual Datasets/cnn_l8/tfrecords/nocnn_BO_4_1_all.tfrecord
Success \o/
Putting 57 observations into nocnn_BO_5_1_all.tfrecord
Observation: 0/57
Observation: 10/57
Observation: 20/57
Observation: 30/57
Observation: 40/57
Observation: 50/57
/Users/robmarty/Google Drive/World Bank/IEs/Pakistan Poverty Estimation/Data/DHS/FinalData/Individual Datasets/cnn_l8/tfrecords/nocnn_BO_5_1_all.tfrecord
Success \o/
Putting 63 observations into nocnn_BU_1_1_all.tfrecord
Observation: 0/63
Observation: 10/63
Observation: 20/63
Observation: 30/63
Observation: 40/63
Observation: 50/63
Observation: 60/63
/Users/robmarty/Google Drive/World Bank/IEs/Pakistan Poverty Estimation/Data/DHS/FinalData/Individual Datasets/cnn_l8/tfrecords/nocnn_BU_1_1_all.tfrecord
Success \o/
Putting 77 observations into nocnn_BU_2_1_all.tfrecord
Observation: 0/77
Observation: 10/77
Observation: 20/77
Observation: 30/7

In [ ]:
for row_i in range(0,10):
    print(row_i)
    proto_examples_i = utils.prep_cnn_np(survey_df_yeari_chunki.iloc[[row_i]], SATELLITE, KERNEL_SIZE, year_i)

In [ ]:
survey_df_yeari_chunki.iloc[[4]]